In [1]:
import numpy as np
import csv

In [2]:
def sigmoid(z):
    f = 1. / (1. + np.exp(-z))
    bound = 1e-8
    return np.clip(f, bound, 1-bound)

In [6]:
X_train_fpath = "data/X_train"
Y_train_fpath = "data/Y_train"
train_x = np.genfromtxt(X_train_fpath, delimiter=',', skip_header=1)
train_y = np.genfromtxt(Y_train_fpath, delimiter=',', skip_header=1)

In [4]:
w = np.zeros(shape = (len(train_x[0]),1))
lr_w = np.zeros((len(train_x[0]), 1))
iteration = 10000

In [6]:
print(train_x.shape)

(32561, 123)


In [16]:
x_t = np.transpose(train_x)

feature_size = len(train_x[0])
train_data_size = len(train_x)

mean1_star = np.zeros((feature_size,))
mean2_star = np.zeros((feature_size,))
sigma1_star = np.zeros((feature_size, feature_size))
sigma2_star = np.zeros((feature_size, feature_size))

count_1 = 0
count_2 = 0

for i in range(train_data_size):
    if train_y[i] == 1:
        count_1 += 1
        mean1_star += train_x[i]
    else:
        count_2 += 1
        mean2_star += train_x[i]
mean1_star = mean1_star / count_1
mean2_star = mean2_star / count_2
for i in range(train_data_size):
    if train_y[i] == 1:
        sigma1_star += np.dot(np.transpose([train_x[i] - mean1_star]), [train_x[i] - mean1_star])
    else:
        sigma2_star += np.dot(np.transpose([train_x[i] - mean2_star]), [train_x[i] - mean2_star])
        
sigma1_star /= count_1
sigma2_star /= count_2
shared_sigma = (float(count_1) / train_data_size) * sigma1_star + (float(count_2) / train_data_size) * sigma2_star

In [50]:
inv_size = len(shared_sigma)
i = np.eye(inv_size,inv_size)
sigma_inverse = np.linalg.lstsq(shared_sigma, i)[0]

w = np.dot((mean1_star - mean2_star), sigma_inverse)
b =(-0.5) * np.dot(np.dot(mean1_star, sigma_inverse), mean1_star) \
    + (0.5) * np.dot(np.dot(mean2_star, sigma_inverse), mean2_star) \
    + np.log(count_1/count_2)


/Users/maxlee/.conda/envs/n_tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  This is separate from the ipykernel package so we can avoid doing imports until


In [49]:
# accuracy
count = 0;
train_x_t = train_x.T
train_y_pred = sigmoid(np.dot(w, train_x_t) + b)

for i in range(len(train_y_pred)):
    if(train_y_pred[i] > 0.5 and train_y[i] == 1):
        count += 1
    elif(train_y_pred[i] < 0.5 and train_y[i] == 0):
        count += 1
print(float(count) / len(train_y))

0.8421731519302232


In [35]:
test_x = np.load("test_x.npy")

In [54]:
#predict
x = test_x.T
a = np.dot(w, x) + b
test_y = sigmoid(a)
test_y_out = []
for i in test_y:
    if i < 0.5:
        test_y_out.append('0')
    else:
        test_y_out.append('1')

In [55]:
output = open("hw2_generative_1", 'w', newline='')
writer = csv.writer(output)
writer.writerow(['id', 'label'])
for i in range(len(test_y_out)):
    writer.writerow([str(i+1), test_y_out[i]])
output.close()